In [1]:
# Homework 01
# Hunter Rogers 
# Machine Learning

In [2]:
import pandas as pd
import numpy
import math

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [4]:
display(train_df)

display(train_df['label'].unique())

len(train_df['label'].unique())

,buying,maint,doors,persons,lug_boot,safety,label
0,low,vhigh,3,4,med,med,unacc
1,low,med,2,more,small,low,unacc
2,vhigh,high,4,4,small,med,unacc
3,low,low,4,2,small,med,unacc
4,low,low,2,4,med,low,unacc
...,...,...,...,...,...,...,...
1377,high,med,2,4,med,low,unacc
1378,low,vhigh,4,4,small,med,unacc
1379,vhigh,vhigh,4,more,big,med,unacc
1380,high,low,2,2,med,med,unacc


array(['unacc', 'good', 'acc', 'vgood'], dtype=object)

4

1. In this problem, we will work with a modified version of the badges data we saw
class. We are given names of the ML researchers along with features, and the task is
to assign a positive or a negative label to their badge. There are four features:


(a) is the length of the first name > 5 (lenFirst): Yes or No

(b) is the length of the last name > 5 (lenLast): Yes or No

(c) is the first letter of the first and last name same (sameFirst): Yes or No

(d) the most frequent vowel that is not ‘u’ (vowel): a, e, i, o

(a) [5 points] How many possible functions are there to map these four features to
a boolean decision? How many functions are consistent with the given training
dataset?

There are 1024 possible functions and only one that fits the data (since the function is created from the data exactly)

In [6]:
options = 2 * 2 * 2 * 4

functions = options * options

functions

1024

(b) [3 points] What is the entropy of the labels in this data? Remember that when
calculating entropy, the base of the logarithm should be 2.

In [ ]:
Name lenFirst lenLast sameFirst vowel Badge
Pieter Bartlett Yes Yes No e +ve
George Berg Yes No No e −ve
Hiroshi Motoda Yes Yes No o +ve
Filippo Neri Yes No No i −ve
Robert Roos Yes No Yes o +ve
Satinder Singh Yes No Yes i +ve
Maja Mataric No Yes Yes a +ve
Arun Sharma No Yes No a −ve
Michael Meystel Yes Yes Yes e −ve

In [5]:
# initialise data of lists.
data = {'Name':['Pieter Bartlett', 'George Berg', 'Hiroshi Motoda', 'Filippo Neri', 'Robert Roos', 'Satinder Singh', 'Maja Mataric', 'Arun Sharma', 'Michael Meystel'],
 'lenFirst':['Yes','Yes','Yes','Yes','Yes','Yes','No','No','Yes'],
 'lenLast':['Yes','No','Yes','No','No','No','Yes','Yes','Yes'],
 'sameFirst':['No','No','No','No','Yes','Yes','Yes','No','Yes'],
 'vowel':['e','e','o','i','o','i','a','a','e'],
 'Badge':['+','-','+','-','+','+','+','-','-']}
 
# Create DataFrame
df = pd.DataFrame(data)

In [47]:
display(df)

,Name,lenFirst,lenLast,sameFirst,vowel,Bedge
0,Pieter Bartlett,Yes,Yes,No,e,+
1,George Berg,Yes,No,No,e,-
2,Hiroshi Motoda,Yes,Yes,No,o,+
3,Filippo Neri,Yes,No,No,i,-
4,Robert Roos,Yes,No,Yes,o,+
5,Satinder Singh,Yes,No,Yes,i,+
6,Maja Mataric,No,Yes,Yes,a,+
7,Arun Sharma,No,Yes,No,a,-
8,Michael Meystel,Yes,Yes,Yes,e,-


In [17]:
def calc_entropy(df, predicted_column):
    if len(df[predicted_column].unique() == 2):
        correct = df[predicted_column].value_counts()[0]
        incorrect = len(df) - correct
        overall_total = len(df)
        if(incorrect == 0):
            overall_entropy = 0
        else:
            overall_entropy = -(correct/overall_total) * math.log((correct/overall_total),2) - (incorrect/overall_total) * math.log((incorrect/overall_total),2)
        for column in df.columns:
            temp_avg_entropy = 0
            for value in df[column].unique():
                temp_df = df[df[column] == value]
                #print(f"{column}: {temp_df[predicted_column].value_counts()[0]}")
                
                temp_correct = temp_df[predicted_column].value_counts()[0]
                temp_incorrect = len(temp_df) - temp_correct
                total = len(temp_df)
                if(temp_incorrect == 0):
                    entropy = 0
                else:
                    entropy = -(temp_correct/total) * math.log((temp_correct/total),2) - (temp_incorrect/total) * math.log((temp_incorrect/total),2)
                print(f"For {value} in {column}: $$H(Badge) = -({temp_correct}/{total})log_2({temp_correct}/{total}) - ({temp_incorrect}/{total})log_2({temp_incorrect}/{total}) = {round(entropy, 2)}$$")
                temp_avg_entropy += (entropy * total)
            temp_avg_entropy = temp_avg_entropy / len(df)
            info_gain = overall_entropy - temp_avg_entropy
            print(info_gain)
            
    else:
        print('nah')



In [12]:
temp_correct = 5
temp_incorrect = 4
total = 9

entropy = -(temp_correct/total) * math.log((temp_correct/total),2) - (temp_incorrect/total) * math.log((temp_incorrect/total),2)

entropy

0.9910760598382222

ValueError: math domain error

In [18]:
calc_entropy(df, 'Badge')

For Pieter Bartlett in Name: $$H(Badge) = -(1/1)log_2(1/1) - (0/1)log_2(0/1) = 0$$
For George Berg in Name: $$H(Badge) = -(1/1)log_2(1/1) - (0/1)log_2(0/1) = 0$$
For Hiroshi Motoda in Name: $$H(Badge) = -(1/1)log_2(1/1) - (0/1)log_2(0/1) = 0$$
For Filippo Neri in Name: $$H(Badge) = -(1/1)log_2(1/1) - (0/1)log_2(0/1) = 0$$
For Robert Roos in Name: $$H(Badge) = -(1/1)log_2(1/1) - (0/1)log_2(0/1) = 0$$
For Satinder Singh in Name: $$H(Badge) = -(1/1)log_2(1/1) - (0/1)log_2(0/1) = 0$$
For Maja Mataric in Name: $$H(Badge) = -(1/1)log_2(1/1) - (0/1)log_2(0/1) = 0$$
For Arun Sharma in Name: $$H(Badge) = -(1/1)log_2(1/1) - (0/1)log_2(0/1) = 0$$
For Michael Meystel in Name: $$H(Badge) = -(1/1)log_2(1/1) - (0/1)log_2(0/1) = 0$$
0.9910760598382222
For Yes in lenFirst: $$H(Badge) = -(4/7)log_2(4/7) - (3/7)log_2(3/7) = 0.99$$
For No in lenFirst: $$H(Badge) = -(1/2)log_2(1/2) - (1/2)log_2(1/2) = 1.0$$
0.0025652873671375698
For Yes in lenLast: $$H(Badge) = -(3/5)log_2(3/5) - (2/5)log_2(2/5) = 0.97$$
F